# Tutorial 4: Memory- and time-efficient solving of ME-models

In this tutorial we will convert the ME-model object to an NLP mathematical representation to save memory and time in simulating many conditions.

## Import libraries

In [1]:
import sys
sys.path.insert(0, '../../')
import coralme
from helpers import get_nlp,optimize

## Load

Load the ME-model coming out of the Troubleshooter

In [2]:
#me = coralme.io.json.load_json_me_model("MEModel-step3-bsubtilis-TS.json")
me = coralme.io.pickle.load_pickle_me_model('../../Zymomonas_mobilis/./outputs/MEModel-step3-zymomonas-TS.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../../Zymomonas_mobilis/./outputs/MEModel-step3-zymomonas-TS.pkl'

## Convert to NLP problem

The ME-model object *me* is a big object containing all data and metadata. This is not necessary when performing large-scale simulations, such as gene knockouts, or growth simulations under hundreds of conditions.

So, in these cases we only need the mathematical problem representing the ME-model, which is *nlp*.

In [ ]:
nlp = get_nlp(me)

## Retrieve metabolite and reaction indexes

The *nlp* now contains the mathematical representation, very similar to a struct object of the COBRA Toolbox in MATLAB. Similarly, reactions and metabolites are now accessed from integer indexes. We can create a dictionary from the original model to map reaction ids to indexes

In [ ]:
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}

From now on, *me* is no longer necessary and can be deleted to save memory usage. This is especially helpful when running parallelized simulations.

In [ ]:
# del me

## Solving the MEModel vs. NLP

Now we can call the modified *optimize* function in *helpers*. This function was modified from the me.optimize() function of a coralme.core.model.MEModel.

Here you can see the speed-up when solving from scratch and solving from the NLP. The speed-up is even more noticeable with bigger models, as lamdifying a longer list of constraints will take much longer.

### ME-model

In [ ]:
%%time
me.optimize(max_mu = 0.1, min_mu = 0., maxIter = 100, lambdify = True,
		tolerance = 1e-6, precision = 'quad', verbose = True)

### NLP

In [ ]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

**Speed-up of complete calculation from ~92 seconds to ~80 seconds!**

## Re-using the basis for even more speed-up

We can re-use a basis from a previously successful simulation to warm-start the first iteration and save even more time! 

### Re-using basis

In [ ]:
%%time
sol,_ = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 1,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

### Cold start

In [ ]:
%%time
sol,_ = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 1, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

**Speed-up of first iteration from ~57 seconds to ~6 seconds!**

### Full calculation

In [ ]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.1, min_mu = 0., maxIter = 100, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

**Speed-up of complete calculation from ~80 seconds to ~33 seconds!**

## Modifying the NLP

As previously mentioned, the NLP resembles a struct object of the COBRA Toolbox. The model is stored as a collection of vectors and matrices representing stoichiometries, bounds and other variables needed by the solvers.

The relevant properties are:
* **xu**: Upper bounds
* **xl**: Lower bounds
* **S**: Stoichiometric matrix (Metabolites x Reactions)

The carbon source right now is Glucose, so we will change its bound to -10 to try to achieve maximum growth rate. 

**Note that bounds contain *lambdify* objects, not floats!**

In [ ]:
nlp.xl[rxn_index_dct["EX_glc_e"]] = lambda x:-10

In [ ]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.5, min_mu = 0., maxIter = 100, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

## Modifying the NLP from a dictionary of new bounds

Make sure to follow this method so that the lambda does not store pointers to a variable but rather a fixed constant (if that is what you want).

In [ ]:
def set_exchanges(nlp,dct):
    for k,v in dct.items():
        nlp.xl[rxn_index_dct[k]] = lambda _,x=v:x

In [ ]:
exchanges = {
    "EX_glc_e" : -10,
    "EX_o2_e" : -0.6,
    "EX_fru_e" : -5,
}

In [ ]:
set_exchanges(nlp,exchanges)

## Inspecting the solution

The function returns a cobra.Solution object just like the one stored in me.solution. For more details inspecting *sol*, refer to Tutorial 3.

In [ ]:
sol